### 다층 퍼셉트론(MultiLayer Perceptron, MLP)
* 구조
    * 다층 퍼셉트론과 단층 퍼셉트론의 차이는 다층 퍼셉트론은 입력층과 출력층 중간에 층을 더 추가하였다는데 이층을 은닉층(Hidden layer)이라고 한다.
    * 다층 퍼셉트론은 중간에 은닉층이 존재한다는 점이 단층 퍼셉트론과 다르다. 다층 퍼셉트론은 줄여서 MLP라고도 부른다
    * XOR 예제에서는 은닉층 1개만으로 문제를 해결할 수 있었지만, 다층 퍼셉트론은 본래 은닉층이 1개 이상인 퍼셉트론을 말한다.
    * XOR 문제보다 더욱 복잡한 문제를 해결하기 위해서 다층 퍼셉트론은 중간에 수많은 은닉층을 더 추가할 수 있다.
    * 은닉층이 2개 이상인 신경망을 심층 신경망(Deep Neural Network, DNN)이라고 한다.

* 딥러닝으로 XOR 문제 해결
    * 1 개의 입력 층, 6개의 노드를 가지는 1개의 은닉 층 그리고 출력 층으로 구성되는 딥러닝 아키텍처를 가정한다.
    * 입력 층과 은닉 층 사이의 가중치는 W2, 은닉 층과 출력 층 사이의 가중치는 W3, 은닉 층의 바이어스는 B2, 그리고 출력 층의 바이어스를 B3로 정의하였다.
    * 은닉 층 노드는 6개로 정의 하였는데 이러한 은닉 층의 노드 수에 대한 정해진 규칙은 없으며 우리가 원하는 만큼 임의의 개수로 만들 수 있는데, 은닉 층 개수와 해당 각 은닉 층에 놓여지는 노드의 개수가 많아지면 학습 속도가 느려지기 때문에 적절한 개수의 은닉 층과 노드를 정의하는것이 중요하다.

In [2]:
import numpy as np

def sigmoid(x):
    return 1. / (1. + np.exp(-x))

def numerical_derivative(f,x):
    delta_x = 1e-4
    grad = np.zeros_like(x)
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        idx = it.multi_index
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + delta_x
        fx1 = f(x)
        
        x[idx] = float(tmp_val) - delta_x
        fx2 = f(x)
        grad[idx] = (fx1 - fx2) / (2*delta_x)
        
        x[idx] = tmp_val
        it.iternext()
    return grad

In [12]:
class LogicGate:
    def __init__(self, gate_name, xdata, tdata):
        self.name = gate_name
        self.xdata = xdata.reshape(4,2)
        self.tdata = tdata.reshape(4,1)
        
        # 입력층 노드 2개, 은닉층 노드 6개. 은닉층 개수는 적당한 값으로 정한다.
        self.w2 = np.random.rand(2,6)
        self.b2 = np.random.rand(6)
        
        #은닉층 노드 6개, 은닉층2 노드 4개
        self.w3 = np.random.rand(6,4)
        self.b3 = np.random.rand(4)
        
        #은닉층2 노드 4개, 출력층 노드 1개
        self.w4 = np.random.rand(4,1)
        self.b4 = np.random.rand(1)
        
        self.learning_rate = 1e-2
        
    def feed_forward(self):
        delta = 1e-7
        z2 = np.dot(self.xdata, self.w2) + self.b2
        a2 = sigmoid(z2)
        z3 = np.dot(a2, self.w3) + self.b3
        
        a3 = sigmoid(z3)
        z4 = np.dot(a3, self.w4) + self.b4
        
        y = a4 = sigmoid(z4)
        return -np.sum(self.tdata * np.log(y+delta)+(1-self.tdata) * np.log((1-y)+delta))
    
    def loss_val(self):
        delta = 1e-7
        z2 = np.dot(self.xdata, self.w2) + self.b2
        a2 = sigmoid(z2)
        z3 = np.dot(a2, self.w3) + self.b3
        
        a3 = sigmoid(z3)
        z4 = np.dot(a3, self.w4) + self.b4
        
        y = a4 = sigmoid(z4)
        
        
        
        return -np.sum(self.tdata*np.log(y+delta)+(1-self.tdata)*np.log((1 - y)+delta))
    
    def train(self):
        f = lambda x: self.feed_forward()
        print('Initial loss value = ', self.loss_val())
        for step in range(35000):
            self.w2 -= self.learning_rate * numerical_derivative(f, self.w2)
            self.b2 -= self.learning_rate * numerical_derivative(f, self.b2)
            self.w3 -= self.learning_rate * numerical_derivative(f, self.w3)
            self.b3 -= self.learning_rate * numerical_derivative(f, self.b3)
            self.w4 -= self.learning_rate * numerical_derivative(f, self.w4)
            self.b4 -= self.learning_rate * numerical_derivative(f, self.b4)
            if (step % 1000 == 0):
                print('step=', step, 'loss value =', self.loss_val())
    
    def predict(self, input_data):
        self.xdata = input_data
        z2 = np.dot(self.xdata, self.w2) + self.b2
        a2 = sigmoid(z2)
        z3 = np.dot(a2, self.w3) + self.b3
        
        a3 = sigmoid(z3)
        z4 = np.dot(a3, self.w4) + self.b4
        
        y = a4 = sigmoid(z4)
        
        if y > 0.5:
            result = 1
        else:
            result = 0
        
        return y,result

        

In [13]:
#XOR 논리 게이트 학습
xdata = np.array([ [0,0],[0,1],[1,0],[1,1] ])
tdata = np.array([0,1,1,0])
xor_obj = LogicGate('XOR',xdata,tdata)
xor_obj.train()

test_data = np.array([[0,0],[0,1],[1,0],[1,1]])
for data in test_data:
    sigmoid_val, logical_val = xor_obj.predict(data)
    print(data, '=', logical_val)

Initial loss value =  4.902762557344618
step= 0 loss value = 4.786076620337502
step= 1000 loss value = 2.7717408300283077
step= 2000 loss value = 2.7711116580680972
step= 3000 loss value = 2.770418715405767
step= 4000 loss value = 2.7696262404548424
step= 5000 loss value = 2.76868483618047
step= 6000 loss value = 2.767521967367411
step= 7000 loss value = 2.766024889008363
step= 8000 loss value = 2.7640079591900863
step= 9000 loss value = 2.761145741982988
step= 10000 loss value = 2.7568258271574457
step= 11000 loss value = 2.749799436496631
step= 12000 loss value = 2.7372937563515456
step= 13000 loss value = 2.7127021844652957
step= 14000 loss value = 2.660227439287938
step= 15000 loss value = 2.5489763793280025
step= 16000 loss value = 2.3603355706158
step= 17000 loss value = 2.1420583296087905
step= 18000 loss value = 1.7660498596955647
step= 19000 loss value = 0.9755134283874983
step= 20000 loss value = 0.4173153809924812
step= 21000 loss value = 0.21416862726234775
step= 22000 loss

### Deep Learning
* 지금까지는 OR,AND,XOR 게이트 등. 퍼셉트론이 가야할 정답을 참고로 퍼셉트론이 정답을 출력할 때까지 가중치를 바꿔보면서 맞는 가중치를 찾았다. 즉, 가중치를 수동으로 찾았다.
* 하지만 이제는 기계가 가중치를 스스로 찾아내도록 자동화시켜야하는데, 이것이 머신 러닝에서 말하는 학습(Training)단계에 해당된다.
* 앞서 선형 회귀와 로지스틱 회귀에서 보았듯이 손실 함수(Loss function)와 옵티마이저(Optimizer)를 사용한다.
* 만약 학습을 시키는 인공 신경망이 심층 신경망일 경우에는 이를 심층 신경망을 학습시킨다고 하여, 딥 러닝(Deep Leaning)이라고 한다.

### ANN or DNN 기초 개념
* 피드 포워드 신경망(Feed-forward Neural Network, FFNN)
    * 다층 퍼셉트론(MLP)과 같이 입력층에서 출력층 방향으로 연산이 전개되는 신경망을 피드 포워드 신경망 이라고 한다.
    * 순환신경망(Recurrent Neural Network, RNN)은 은닉층의 출력값을 출력층으로도 값을 보내지만, 동시에 은닉층의 출력값이 다시 은닉층의 입력으로 사용된다.
* 전결합층(Fully-Connected Layer, FC, Dense Layer)
    * 다층 퍼셉트론은 은닉층과 출력층에 있는 모든 뉴런은 바로 이전 층의 모든 뉴런과 연결돼있다. 그와 같이 어떤 층의 모든 뉴런이 이전 층의 모든 뉴런과 연결돼 있는 층을 전결합층이라고 하며, 줄여서 FC라고 부르기도 한다.
    * 다층 퍼셉트론의 모든 은닉층과 출력층은 전결합층이다. 이와 동일한 의미로 밀집층(Dense layer)이라고 부르기도 하는데, 케라스에서는 밀집층을 구현할 때 Dense()를 사용한다.

* 활성화 함수(Activation Function)
    * 은닉층과 출력층의 뉴런에서 출력값을 결정하는 함수를 활성화 함수(Activation Function)라고 하는데 계단 함수는 이러한 활성화 함수의 하나이다.
    * 활성화 함수의 특징
        - 선형 함수가 아닌 비선형 함수여야 한다. 선형 함수란 출력이 입력의 상수배만큼 변하는 함수를 선형함수라고 하는데 예를 들어 f(x) = Wx+b 라는 함수가 있을 때, W와 b는 상수이다. 이 식은 그래프를 그리면 직선이 그려진다. 반대로 비선형 함수는 직선 1개로는 그릴 수 없는 함수를 말한다.
        - 인공 신경망에서 활성화 함수는 반드시 비선형 함수여야 한다. 앞서 퍼셉트론에서도 계단 함수라는 활성화 함수를 사용했는데 계단 함수 또한 비선형 함수에 속한다.

#### 활성화 함수
* 계단 함수(Step Function)
    - 계단 함수는 이제 거의 사용되지 않지만, 퍼셉트론을 통해 처음으로 인공 신경망을 배울 때 가장 처음 접하게 되는 활성화 함수이다.
    - 값이 0 또는 1로 고정되어 있으므로 기울기가 항상 0이다. 또, x가 0인 지점에서는 미분이 불가능하다.

* 시그모이드 함수(Sigmoid Function)와 기울기 소실
    - 시그모이드 함수를 사용한 어떤 인공 신경망이 있다고 가정해보자
    - 우선 인공 신경망은 입력에 대해서 순전파 연산을 하고, 그리고 순전파 연산을 통해 나온 예측값과 실제값의 오차를 손실 함수(loss function)을 통해 계산하고, 그리고 이 손실을 미분을 통해서 기울기를 구하고, 이를 통해 역전파를 수행한다.
    - 시그모이드 함수의 출력값이 0 또는 1에 가까워지면 그래프의 기울기가 완만해진다.
    - 시그모이드 함수의 문제점은 미분을 해서 기울기를 구할 때 발생한다. 기울기를 계산하면 0에 가까운 아주 작은 값이 나오게 된다. 그런데 역전파 과정에서 0에 가까운 아주 작은 기울기가 곱해지게 되면, 앞단에는 기울기가 잘 전달되지 않게 된다. 이러한 현상을 기울기 소실 문제라고 한다.

* 하이퍼볼릭탄젠트 함수(Hyperbolic tangent function)
    - 하이퍼볼릭탄젠트 함수(tanh)는 입력값을 -1과 1사이의 값으로 변환한다.
    - 하이퍼볼릭탄젠트 함수도 -1과 1에 가까운 출력값을 출력할 때, 시그모이드 함수와 같은 문제가 발생하지만 하이퍼볼릭탄젠트 함수의 경우에는 시그모이드 함수와는 달리 0을 중심으로 하고 있는데, 이때문에 시그모이드 함수와 비교하면 반환값의 변화폭이 더 크다. 그래서 시그모이드 함수보다는 기울기 소실 증상이 적은 편이기 때문에 은닉층에서 시그모이드 함수보다는 많이 사용된다.

* 렐루 함수(ReLU)
    - 인공 신경망에서 가장 최고의 인기를 얻고 있는 함수이고 수식은 f(x) = max(0,x)로 아주 간단하다.
    - 렐루 함수는 음수를 입력하면 0을 출력하고, 양수를 입력하면 입력값을 그대로 반환한다. 렐루 함수는 특정 양수값에 수렴하지 않으므로 깊은 신경망에서 시그모이드 함수보다 훨씬 더 잘 작동한다. 뿐만아니라, 렐루 함수는 시그모이드 함수와 하이퍼볼릭탄젠트 함수와 같이 어떤 연산이 필요한 것이 아니라 단순 임계값이므로 연산 속도도 빠르다.
    - 하지만 문제는 입력값이 음수면 기울기도 0이 된다. 그리고 이 뉴런은 다시 회생한느 것이 매우 어렵다. 이 문제를 죽은 렐루 (Dying ReLU)라고 한다.

* 리키 렐루(Leaky ReLU)
    - Leaky ReLU는 입력값이 음수일 경우에 0이 아니라 0.001과 같은 매우 작은 수를 반환하도록 되어있다.
    - 수식은 f(x)=max(ax,x)로 아주 간단하며. a는 하이퍼파라미터로 Leaky(새는) 정도를 결정하며 일반적으로는 0.01의 값을 가진다. 여기서 말하는 '새는 정도'라는 것은 입력값이 음수일 때의 기울기를 비유하고 있다.
    - 위의 그래프에서는 새는 모습을 확실히 보여주기 위해 a를 0.1로 잡았다. 위와 같이 입력값이 음수라도 기울기가 0이 되지 않는다면 ReLU는 죽지 않는다.

* 소프트맥스 함수(Softmax function)
    - 은닉층에서 ReLU(또는 ReLU)함수들을 사용하는 것이 일반적이지만 분류 문제는 로지스틱 회귀와 소프트맥스 회귀를 출력층에 적용하여 사용한다.
    - 소프트맥스 함수는 시그모이드 함수처럼 출력층의 뉴런에서 주로 사용된다.
    - 시그모이드 함수는 두 가지 중 하나를 고르는 이진 분류 문제에 사용되고 세가지 이상의 선택지 중 하나를 고르는 다중 클래스 분류 문제에는 소프트맥스 함수가 주로 사용된다.